In [15]:
# Importing the required packages
import numpy as np
import pandas as pd
import tensorflow as tf

In [16]:
data = pd.read_csv('fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [17]:
# Separating the data into labels and features
labels, data = data['emotion'], data.drop(['emotion','Usage'], axis=1)
data.head()

(35887, 1)

In [21]:
# Taking in the pixels of image i.e the features as a numpy array
pixel_data = data.values
pixel_data = pixel_data.astype(str)
# Here we know that the images are one-dimensional(grayscale) since the depth of the matrix 1 of every element
pixel_data = np.core.defchararray.rsplit(pixel_data, sep=None, maxsplit=None)
pixel_data.shape

(35887, 1)

In [ ]:
# Now let's set aside the image data and process the labels. Since labels are categorical variables we can one-hot-encode them
encoded_labels = pd.get_dummies(labels)
encoded_labels = encoded_labels.as_matrix()
encoded_labels.shape

In [ ]:
# Defining tensors for inputs and labels 

# Placeholder tensor for inputs. In this case, input is a 28x28 image.
input_image_data = tf.placeholder(tf.float32,shape=[None,28,28,1])

# Placeholder for labels 
label_data = tf.placeholder(tf.float32, shape=[None,7])

keep_prob = tf.placeholder(tf.float32,shape=None)

In [ ]:
# Now let's start off by building the network
def build_cnn(inputs, labels):
    
    # Input layer of conv-net
    input_layer = input_image_data
    
    # First convolutional layer
    neural_net = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    # First pooling layer
    neural_net =  tf.layers.max_pooling2d(inputs=neural_net, pool_size=[2, 2], strides=2)
    
    # Second convolutional layer 
    neural_net = tf.layers.conv2d(
      inputs=neural_net,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    # Second pooling layer
    neural_net = tf.layers.max_pooling2d(inputs=neural_net, pool_size=[2,2], strides=2)
    
    # Flattening layer
    neural_net = tf.reshape(neural_net, [-1, 7 * 7 * 64])
    
    # Two stacked Fully connected layers
    neural_net = tf.layers.dense(inputs=neural_net, units=1024, activation=tf.nn.relu)
    neural_net = tf.layers.dropout(inputs=neural_net, rate=0.4)
    
    neural_net = tf.layers.dense(inputs=neural_net, units=1024, activation=tf.nn.relu)
    neural_net = tf.layers.dropout(inputs=neural_net, rate=0.4)
    
    # Output Layer
    logits = tf.layers.dense(inputs=neural_net, units=7)
    
    return logits 

In [ ]:
# Calling the build_cnn function 
cnn_built = build_cnn(input_image_data, label_data)
cnn_built = tf.identity(cnn_built, name='cnn')

# Loss and optimizer 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=cnn_built, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [ ]:
# Two functions to train the network and print stats 
def train_network(session, optimizer, keep_probability, feature_batch, label_batch):
    
    # Optimizing the neural net
    session.run(optimizer, feed_dict={x:feature_batch,y:label_batch, keep_prob:keep_probability})
    pass

def print_stats(session, feature_batch, label_batch, cost, accuracy):
    
    # Calculating loss and validation accuracy
    loss = sess.run(cost, feed_dict={x: feature_batch, y: label_batch, keep_prob: 1.})
    valid_acc = sess.run(accuracy, feed_dict={
                x: valid_features,
                y: valid_labels,
                keep_prob: 1.})
    print("loss", loss, "valid_acc", valid_acc)
    pass    

In [ ]:
# Defining the hyperparameters
epochs = 30
batch_size = 128
keep_prob = 0.5

In [ ]:
# Finally let's get training 
print("Training on the data")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        train_network(session,optimizer, keep_probability, feature_batch, label_batch)
    
    print('Epoch {:>2}:  '.format(epoch + 1), end='')
    print_stats(sess, batch_features, batch_labels, cost, accuracy)